In [112]:
import pandas as pd
df= pd.read_csv('titanic.csv')
df

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,"Montvila, Rev. Juozas",2,male,27.0,0,0,211536,13.0000,NaN,S,0
887,888,"Graham, Miss. Margaret Edith",1,female,19.0,0,0,112053,30.0000,B42,S,1
888,889,"Johnston, Miss. Catherine Helen ""Carrie""",3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0
889,890,"Behr, Mr. Karl Howell",1,male,26.0,0,0,111369,30.0000,C148,C,1


In [113]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()
df.isnull().sum()
df.Age = df.Age.fillna(df.Age.mean())
df.isnull().sum()
df.describe()

,Pclass,Age,Fare,Survived
count,891.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,32.204208,0.383838
std,0.836071,13.002015,49.693429,0.486592
min,1.000000,0.420000,0.000000,0.000000
25%,2.000000,22.000000,7.910400,0.000000
50%,3.000000,29.699118,14.454200,0.000000
75%,3.000000,35.000000,31.000000,1.000000
max,3.000000,80.000000,512.329200,1.000000


In [117]:
df['Fare'].skew

<bound method Series.skew of 1      71.2833
2       7.9250
3      53.1000
4       8.0500
5       8.4583
        ...   
886    13.0000
887    30.0000
888    23.4500
889    30.0000
890     7.7500
Name: Fare, Length: 655, dtype: float64>

In [114]:
df['Fare'].skew()
percentile25= df['Age'].quantile(0.25)
percentile75=df['Age'].quantile(0.75)
iqr =percentile75-percentile25
upper_limit= percentile75 + 1.5 * iqr
lower_limit= percentile25 - 1.5 * iqr
df=df[df['Age']< upper_limit]
df= df [df['Age'] > lower_limit]
df.describe()


upper_limit= df.Fare.quantile(0.9)
lower_limit= df.Fare.quantile(0.1)
print (lower_limit)
df= df[df['Fare']< upper_limit]
df= df [df['Fare'] > lower_limit]


7.4958


In [115]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [116]:
#dummies = pd.get_dummies(inputs.Sex)
#dummies.head(3)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
inputs['Sex'] = le.fit_transform(inputs['Sex'])

inputs = pd.DataFrame({'Sex': inputs['Sex']})
inputs

,Sex
1,female
2,female
3,female
4,male
5,male
...,...
886,male
887,female
888,female
889,male


In [111]:
#inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Sex
1,0
2,0
3,0


In [95]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

KeyError: "['Sex', 'male'] not found in axis"

In [65]:
inputs.describe()

,Pclass,Age,Fare
count,655.000000,655.000000,655.000000
mean,2.458015,28.445917,20.440565
std,0.736339,10.178497,16.055977
min,1.000000,3.000000,7.520800
25%,2.000000,22.000000,8.050000
50%,3.000000,29.699118,13.416700
75%,3.000000,32.750000,26.287500
max,3.000000,54.000000,76.291700


In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)

X_test=sc.transform(X_test)


In [67]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [68]:
model.fit(X_train,y_train)

GaussianNB()

In [69]:
model.score(X_test,y_test)

0.7614213197969543

In [72]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.69565217, 0.69565217, 0.70652174, 0.76923077, 0.73626374])

In [73]:
from sklearn.ensemble import RandomForestClassifier
test1=RandomForestClassifier(n_estimators=100)
test1.fit(X_train,y_train)

RandomForestClassifier()

In [74]:
test1.score(X_test,y_test)

0.8375634517766497

In [75]:
from sklearn.tree import DecisionTreeClassifier
test2= DecisionTreeClassifier()
test2.fit(X_train,y_train)

DecisionTreeClassifier()

In [76]:
test2.score(X_test,y_test)

0.8020304568527918

In [77]:
from sklearn.svm import SVC
test3=SVC(kernel='linear')
test3.fit(X_train,y_train)

SVC(kernel='linear')

In [78]:
test3.score(X_test,y_test)

0.7969543147208121

In [79]:
from sklearn.linear_model import LogisticRegression
test4= LogisticRegression()
test4.fit(X_train,y_train)
test4.score(X_test,y_test)

0.817258883248731